In [ ]:
import numpy as np
import pandas as pd
# build the sequence data
original_data = pd.read_csv('Practice_Log_Demographics.csv')

c:\users\26981\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
# build the sequence data
import datetime
user_id_key = list(original_data['user_id.x'].unique())

In [ ]:
len(user_id_key)

327

In [ ]:
import warnings 
import gc

In [ ]:
original_data = original_data.sort_values(by = ['start_practice'])

In [ ]:
# data preprocessing
# according to sub_chapter_label and user_id, generate sequences
# generate (chapter,sub_chapter,user_id) sequences according to user_id and term
original_data['user_id.x'] = original_data['user_id.x'].astype(int)
original_data['user_id.x'] = original_data['user_id.x'].astype(str)
original_data['user_id.x'] = original_data['term'].str.cat(original_data['user_id.x'], sep=':')
original_data['user_id.x'] = original_data['user_id.x'].astype('category')
original_data['term'] = original_data['term'].astype('category')
original_data['chapter_label'] = original_data['chapter_label'].astype('category')
original_data['sub_chapter_label'] = original_data['sub_chapter_label'].astype('category')

In [ ]:
original_data['start_practice'] = pd.to_datetime(original_data['start_practice'], format = '%Y-%m-%d %H:%M:%S')
original_data['end_practice'] = pd.to_datetime(original_data['end_practice'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
original_data['label'] = original_data['chapter_label'].astype(str) + '_' + original_data['sub_chapter_label'].astype(str)

In [ ]:
user_id_key = list(original_data['user_id.x'].unique())

In [ ]:
len(user_id_key)

1110

In [ ]:
user_record_dict = {}
for item in user_id_key:
    user_record_dict[item] = {}

In [ ]:
for index, row in original_data.iterrows():
    user_id = row['user_id.x']
    if row['label'] not in user_record_dict[user_id].keys():
        user_record_dict[user_id][row['label']] = []
        user_record_dict[user_id][row['label']].append([row['q'],row['label'],row['start_practice'],row['end_practice']])
    else:
        user_record_dict[user_id][row['label']].append([row['q'],row['label'],row['start_practice'],row['end_practice']])

In [ ]:
# construct the record according to students.
user_training_set = {}
for item in user_id_key:
    user_training_set[item] = []
    user_record_key = user_record_dict[item].keys()
    for mid_item in user_record_key:
        mid_sequence = []
        for iter_item in user_record_dict[item][mid_item]:
            mid_sequence.append(iter_item[5])
        user_training_set[item].append(mid_sequence)    

In [ ]:
# train the model and record the params according to students
import hmmlearn.hmm as hmm

states = ['NotUnderstood', 'Understood']
obs = [0,1]
n_states = len(states)
user_model = {}
count = 0
user_model_params = {}
for item in user_id_key:
    user_model_params[item] = {}
    model = hmm.MultinomialHMM(n_components=n_states)
    len_training_set = int(0.8*len(user_training_set[item]))
    mid_training_set = user_training_set[item][:len_training_set]
    mid_test_set = user_training_set[item][len_training_set:]
    all_X = []
    length = []
    start_probability = 0
    for mid_item in mid_training_set:
        if mid_item[0] == 0:
            start_probability += 1
        if len(all_X) == 0:
            well_formed_mid_item = []
            for iter_item in mid_item:
                well_formed_mid_item.append([iter_item])
            all_X = well_formed_mid_item
            length.append(len(mid_item))
        else:
            well_formed_mid_item = []
            for iter_item in mid_item:
                well_formed_mid_item.append([iter_item])
            all_X = np.concatenate([all_X, well_formed_mid_item])
            length.append(len(mid_item))
        #else:
        #   all_X = np.concatenate([all_X, mid_item])
        #    length.append(len(mid_item))
    start_probability = start_probability / len(length)
    start_probability = np.array([start_probability, 1-start_probability])
    #odel.startprob_ = start_probability
    #rint(all_X)
    model.fit(all_X, lengths = length)
    user_model_params[item]["startprob_"] = model.startprob_
    user_model_params[item]["transmat_"] = model.transmat_
    user_model_params[item]["emissionprob_"] = model.emissionprob_
    user_model_params[item]["test_set"] = mid_test_set
    #rint(model.startprob_)
    #rint(model.transmat_)
    #rint(model.emissionprob_)

Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Fitting a model with 5 free scalar parameters with only 1 data points will result in a degenerate solution.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


In [ ]:
# predict and calculate auc and acc
all_count = 0
prob = []
tag = []
for item in user_id_key:
    mid_test_set = user_model_params[item]["test_set"]
    model = hmm.MultinomialHMM(n_components=n_states)
    model.startprob_ = user_model_params[item]["startprob_"]
    model.transmat_ = user_model_params[item]["transmat_"]
    model.emissionprob_ = user_model_params[item]["emissionprob_"]
    for mid_item in user_model_params[item]["test_set"]:
        count = 0
        mid_prob = 0
        while count < len(mid_item):
            #print(int(mid_item[count]))
            if count == 0:
                mid_prob = model.startprob_[0]
            #print(mid_prob)
            #print(mid_item[count])
            #print(model.startprob_)
                mid_tag = 0
                if mid_prob > 0.5:
                    mid_tag = 0
                else:
                    mid_tag = 1
                    mid_prob = 1 - mid_prob
                if mid_tag == mid_item[count]:
                    mid_tag = 1
                else:
                    mid_tag = 0
                prob.append(mid_prob)
                tag.append(mid_tag)
            else:
                #rint(count)
                #rint(mid_item[:count])
                mid_mid_item = mid_item[:count]
                well_formed_mid_item = []
                for iter_item in mid_mid_item:
                    well_formed_mid_item.append([iter_item])
                state_sequence = model.predict(well_formed_mid_item)
                mid_prob = model.transmat_[state_sequence[-1], :]
                mid_prob = mid_prob.dot(model.emissionprob_)
                mid_prob = mid_prob[0]
                mid_tag = 0
                if mid_prob > 0.5:
                    mid_tag = 0
                else:
                    mid_tag = 1
                    mid_prob = 1 - mid_prob
                if mid_tag == mid_item[count]:
                    mid_tag = 1
                else:
                    mid_tag = 0
                prob.append(mid_prob)
                tag.append(mid_tag)
            count += 1
            all_count += 1

In [ ]:
#f = list(zip(prob,tag))
#print(f)

In [ ]:
# calculate acc
sum(tag) / len(tag)

0.6500930747025656

In [ ]:
type(model.transmat_[state_sequence[-1], :])

numpy.ndarray

In [ ]:
# calculate auc
def calAUC(prob,labels):
    f = list(zip(prob,labels))
    rank = [values2 for values1,values2 in sorted(f,key=lambda x:x[0])]
    values = [values1 for values1,values2 in sorted(f,key=lambda x:x[0])]
    rankList = [i+1 for i in range(len(rank)) if rank[i]==1]
    posNum = 0
    negNum = 0
    for i in range(len(labels)):
        if(labels[i]==1):
            posNum+=1
        else:
            negNum+=1
    auc = 0
    auc = (sum(rankList)- (posNum*(posNum+1))/2)/(posNum*negNum)
    return auc

auc = calAUC(prob, tag)
print(auc)

0.656455447558408
